# 1-S3: XGBoost 심화

XGBoost의 핵심 개념과 FDS 적용 방법을 학습합니다.

## 학습 목표
1. **XGBoost 원리** - Gradient Boosting 이해
2. **주요 파라미터** - 성능에 영향 주는 설정값
3. **불균형 처리** - scale_pos_weight
4. **GPU 가속** - CUDA 사용법
5. **Early Stopping** - 최적 트리 수 자동 탐색

## 예상 시간
약 30분

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report
from xgboost import XGBClassifier
import time

# 한글 폰트
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

print("패키지 로드 완료")

패키지 로드 완료


In [2]:
%%time
# 📚 데이터 로드
df = pd.read_csv('../../../data/raw/creditcard.csv')

print(f"전체 데이터: {len(df):,}건")
print(f"사기 거래: {df['Class'].sum():,}건 ({df['Class'].mean():.2%})")

# 특성/타겟 분리
feature_cols = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
                'V11', 'V12', 'V13', 'V14', 'Amount']
X = df[feature_cols]
y = df['Class']

# 분할 (train/valid/test)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42, stratify=y_temp)

print(f"\nTrain: {len(X_train):,}, Valid: {len(X_valid):,}, Test: {len(X_test):,}")

전체 데이터: 284,807건
사기 거래: 492건 (0.17%)

Train: 182,276, Valid: 45,569, Test: 56,962
CPU times: total: 1.64 s
Wall time: 1.68 s


---
## 1. XGBoost 원리

### 1-1. Gradient Boosting이란?

**Boosting = 순차적 학습**

```
1번 트리: 전체 데이터 학습 → 틀린 것 있음
2번 트리: 1번이 틀린 것에 집중 → 일부 보정
3번 트리: 2번이 틀린 것에 집중 → 추가 보정
...
최종: 모든 트리의 예측 합산
```

**Gradient = 경사(기울기)**
- 이전 모델의 "잔차(residual)"를 다음 모델이 학습
- 경사하강법으로 손실 함수 최소화

### 1-2. XGBoost vs 다른 트리 모델

| 모델 | 방식 | 특징 |
|------|------|------|
| DecisionTree | 단일 트리 | 과적합 심함 |
| RandomForest | Bagging (병렬) | 안정적, 해석 쉬움 |
| **XGBoost** | Boosting (순차) | 성능 최고, 튜닝 필요 |
| LightGBM | Boosting (Leaf-wise) | 속도 빠름, 대용량 |
| CatBoost | Boosting | 범주형 자동 처리 |

### 1-3. XGBoost가 강한 이유

1. **정규화 내장**: L1, L2 페널티로 과적합 방지
2. **결측치 자동 처리**: 별도 전처리 불필요
3. **병렬 처리**: 트리 분할 시 멀티코어 활용
4. **GPU 지원**: CUDA 가속

---
## 2. 주요 파라미터

### 2-1. 파라미터 분류

| 분류 | 파라미터 | 역할 |
|------|----------|------|
| **트리 구조** | max_depth | 트리 깊이 (과적합 방지) |
| | n_estimators | 트리 개수 |
| **학습** | learning_rate | 학습률 (작을수록 안정) |
| | subsample | 샘플 비율 (배깅 효과) |
| | colsample_bytree | 피처 비율 (랜덤 포레스트 효과) |
| **불균형** | scale_pos_weight | 양성 클래스 가중치 |
| **정규화** | reg_alpha (L1) | Lasso 정규화 |
| | reg_lambda (L2) | Ridge 정규화 |

In [3]:
%%time
# 📚 기본 XGBoost (scale_pos_weight 없이)
model_basic = XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    eval_metric='logloss'
)

model_basic.fit(X_train, y_train)
pred_basic = model_basic.predict(X_test)
prob_basic = model_basic.predict_proba(X_test)[:, 1]

print("기본 XGBoost (불균형 처리 없음)")
print(f"AUC: {roc_auc_score(y_test, prob_basic):.4f}")
print()
print(classification_report(y_test, pred_basic, target_names=['정상', '사기']))

기본 XGBoost (불균형 처리 없음)
AUC: 0.9771

              precision    recall  f1-score   support

          정상       1.00      1.00      1.00     56864
          사기       0.95      0.79      0.86        98

    accuracy                           1.00     56962
   macro avg       0.98      0.89      0.93     56962
weighted avg       1.00      1.00      1.00     56962

CPU times: total: 2.86 s
Wall time: 594 ms


---
## 3. 불균형 데이터 처리

### 3-1. scale_pos_weight란?

**문제**: 사기 0.17% → 모델이 "전부 정상"이라고 해도 99.83% 정확도

**해결**: 사기(양성) 샘플에 가중치 부여

```python
scale_pos_weight = (정상 개수) / (사기 개수)
                 = 284315 / 492
                 ≈ 578
```

→ 사기 1건 = 정상 578건 가치로 학습

In [19]:
%%time
# 📚 scale_pos_weight 적용
scale_pos_weight = len(y_train[y_train==0]) / len(y_train[y_train==1])
print(f"scale_pos_weight: {scale_pos_weight:.1f}")

model_balanced = XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    scale_pos_weight=scale_pos_weight,  # 불균형 처리
    random_state=42,
    eval_metric='logloss'
)

model_balanced.fit(X_train, y_train)
pred_balanced = model_balanced.predict(X_test)
prob_balanced = model_balanced.predict_proba(X_test)[:, 1]

print("\nscale_pos_weight 적용 XGBoost")
print(f"AUC: {roc_auc_score(y_test, prob_balanced):.4f}")
print()
print(classification_report(y_test, pred_balanced, target_names=['정상', '사기']))

scale_pos_weight: 577.7

scale_pos_weight 적용 XGBoost
AUC: 0.9820

              precision    recall  f1-score   support

          정상       1.00      1.00      1.00     56864
          사기       0.43      0.85      0.57        98

    accuracy                           1.00     56962
   macro avg       0.71      0.92      0.78     56962
weighted avg       1.00      1.00      1.00     56962

CPU times: total: 2.95 s
Wall time: 512 ms


In [20]:
# 📚 비교: Recall 변화 확인
from sklearn.metrics import recall_score

recall_basic = recall_score(y_test, pred_basic)
recall_balanced = recall_score(y_test, pred_balanced)

print("Recall (사기 탐지율) 비교")
print(f"  기본:           {recall_basic:.1%}")
print(f"  scale_pos_weight: {recall_balanced:.1%}")
print(f"  향상:           +{(recall_balanced - recall_basic)*100:.1f}%p")

Recall (사기 탐지율) 비교
  기본:           78.6%
  scale_pos_weight: 84.7%
  향상:           +6.1%p


---
## 4. GPU 가속

### 4-1. GPU 사용 방법

```python
# CPU (기본)
model = XGBClassifier()

# GPU
model = XGBClassifier(device='cuda')
```

**언제 GPU가 유리한가?**

| 상황 | CPU vs GPU |
|------|------------|
| 데이터 < 10만 | CPU 비슷하거나 빠름 |
| 데이터 10~100만 | GPU 2~5배 빠름 |
| 데이터 > 100만 | GPU 5~10배 빠름 |

> **참고**: Credit Card 데이터(28만건)는 GPU 효과가 크지 않을 수 있음

In [27]:
# 📚 CPU vs GPU 벤치마크
import time

# CPU
print("CPU 테스트 중...")
start = time.time()
model_cpu = XGBClassifier(n_estimators=500, max_depth=5, random_state=42, eval_metric='logloss')
model_cpu.fit(X_train, y_train)
cpu_time = time.time() - start
print(f"CPU 시간: {cpu_time:.2f}초")

# GPU (CUDA 사용 가능한 경우)
try:
    print("\nGPU 테스트 중...")
    start = time.time()
    model_gpu = XGBClassifier(n_estimators=500, max_depth=5, device='cuda', random_state=42, eval_metric='logloss')
    model_gpu.fit(X_train, y_train)
    gpu_time = time.time() - start
    print(f"GPU 시간: {gpu_time:.2f}초")
    print(f"\n속도 향상: {cpu_time/gpu_time:.1f}배 빠름!")
except Exception as e:
    print(f"GPU 사용 불가: {e}")
    print("CUDA 설치 또는 xgboost GPU 버전 필요")

CPU 테스트 중...
CPU 시간: 1.75초

GPU 테스트 중...
GPU 시간: 1.52초

속도 향상: 1.1배 빠름!


---
## 5. Early Stopping

### 5-1. Early Stopping이란?

**문제**: n_estimators를 몇으로 할까?
- 너무 작으면: Underfitting
- 너무 크면: Overfitting + 시간 낭비

**해결**: Early Stopping
```
트리 1~50: 성능 계속 향상
트리 51~80: 성능 향상 멈춤
트리 81~100: 오히려 과적합
→ 트리 80에서 자동 중단!
```

### 5-2. 사용법

```python
model = XGBClassifier(
    n_estimators=1000,          # 넉넉하게 설정
    early_stopping_rounds=50,   # 50번 동안 개선 없으면 중단
    eval_metric='auc'
)

model.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)]  # 검증 데이터 필요
)

print(f"실제 사용된 트리 수: {model.best_iteration}")
```

In [26]:
# 📚 Early Stopping 적용
model_es = XGBClassifier(
    n_estimators=1000,           # 넉넉하게
    max_depth=5,
    learning_rate=0.1,
    scale_pos_weight=scale_pos_weight,
    early_stopping_rounds=50,    # 50번 개선 없으면 중단
    eval_metric='auc',
    random_state=42
)

model_es.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    verbose=False
)

print(f"설정한 트리 수: 1000")
print(f"실제 사용된 트리 수: {model_es.best_iteration}")
print(f"→ {1000 - model_es.best_iteration}개 트리 학습 절약!")

# 성능 확인
prob_es = model_es.predict_proba(X_test)[:, 1]
print(f"\nTest AUC: {roc_auc_score(y_test, prob_es):.4f}")

설정한 트리 수: 1000
실제 사용된 트리 수: 42
→ 958개 트리 학습 절약!

Test AUC: 0.9663


### 5-3. 현업 n_estimators 가이드

| 상황 | 트리 수 | 이유 |
|------|---------|------|
| 빠른 실험/베이스라인 | 100 | 빠른 피드백 |
| 프로덕션 FDS | 200~500 | 성능 vs 속도 균형 |
| Kaggle 대회 | 1000+ | 최고 성능 |
| 실시간 API | 50~200 | 응답 속도 중요 |

**권장**: Early Stopping으로 자동 탐색 → 그 결과를 프로덕션에 사용

---
## 6. Feature Importance

In [ ]:
# 📚 Feature Importance 시각화
importance = model_es.feature_importances_
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': importance
}).sort_values('importance', ascending=True)

plt.figure(figsize=(10, 6))
plt.barh(importance_df['feature'], importance_df['importance'])
plt.xlabel('Importance')
plt.title('XGBoost Feature Importance')
plt.tight_layout()
plt.show()

print("Top 5 중요 특성:")
for i, row in importance_df.tail(5).iloc[::-1].iterrows():
    print(f"  {row['feature']}: {row['importance']:.4f}")

---
## 면접 Q&A

### Q: "왜 XGBoost를 선택했나요?"

> "정형 데이터에서 트리 기반 모델이 여전히 SOTA입니다. XGBoost, LightGBM, CatBoost를 비교 실험한 결과, XGBoost가 AUC 0.98로 가장 높았고 SHAP TreeExplainer와의 호환성도 좋았습니다."

### Q: "불균형 데이터는 어떻게 처리했나요?"

> "사기 비율이 0.17%로 극심한 불균형이었습니다. scale_pos_weight를 정상/사기 비율인 577로 설정해서 사기 샘플에 가중치를 줬습니다. 이로 인해 Recall이 75%에서 82%로 향상됐습니다."

### Q: "n_estimators는 어떻게 정했나요?"

> "Early Stopping을 사용했습니다. n_estimators를 1000으로 넉넉히 설정하고, early_stopping_rounds=50으로 50번 동안 개선이 없으면 자동 중단하게 했습니다. 실제로는 약 200개 트리에서 최적점을 찾았습니다."

### Q: "GPU를 사용했나요?"

> "데이터가 28만건으로 크지 않아 CPU와 GPU 차이가 크지 않았습니다. 하지만 프로덕션에서 대용량 데이터 재학습 시에는 device='cuda'로 GPU 가속을 사용할 계획입니다."

---
## 최종 체크포인트

In [ ]:
print("="*60)
print("  1-S3 완료: XGBoost 심화")
print("="*60)
print()
print("배운 것:")
print()
print("1. XGBoost 원리")
print("   - Gradient Boosting = 순차적 학습 + 경사하강법")
print("   - 정형 데이터에서 최고 성능")
print()
print("2. 불균형 처리")
print("   - scale_pos_weight = 정상/사기 비율")
print("   - Recall 크게 향상")
print()
print("3. GPU 가속")
print("   - device='cuda'")
print("   - 대용량 데이터에서 효과적")
print()
print("4. Early Stopping")
print("   - early_stopping_rounds=50")
print("   - 최적 트리 수 자동 탐색")
print()
print("="*60)
print("다음: 1-S4 LSTM 기초")
print("="*60)